<a href="https://colab.research.google.com/github/KinoshitaYstr/mmi-learning/blob/master/chap05v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![代替テキスト](https://)## chap05 ニューラルネットワーク

1.   データが置かれたGoogleドライブをマウント（Colabのメニューで操作）


> ファイル -> ドライブをマウント -> 実行可能なセルが現れる -> ログインして認証コードを入力




2.   /content/driveにマウントされる



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


GiNZAのインストール  

In [2]:
!pip install -U ginza

     |████████████████████████████████| 54.9MB 52kB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 44.8MB 1.9MB/s 
     |████████████████████████████████| 481kB 47.8MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=06af9d64fe7bf4ff00c0d780f22a407b0c24894be4c7f52361cbbceb32e40085
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=e90948b4959417c6dfd93d7f549c7b887d3c6cda874590cb3178a34011b8b2f3
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for ja-ginza-dict: filename=ja_ginza_dict-3.1.0-cp36-none-any.whl size=70877544 sha256=3941b63b35ecffb8a1a98b241e55daf959cebfaf8478e657c5b9f075005e59fc
  Stored in directory: /root/.cache/pip/wheels/30/88/d7/7f0692ba26060966af34538e1079438d1

Lightningのインストール

In [3]:
!pip install pytorch-lightning
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 256kB 4.3MB/s 
     |████████████████████████████████| 829kB 14.6MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=26dc4f34989fe8191d3ea6426e38e19734e581dfc55533d1fd801030ab4a15ef
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

GiNZAの使用準備

In [0]:
import spacy
nlp = spacy.load('ja_ginza')

京都大学情報学研究科--NTTコミュニケーション科学基礎研究所 共同研究ユニット 
 [解析済みブログコーパス](http://nlp.ist.i.kyoto-u.ac.jp/kuntt/#ga739fe2)
の読み込み

In [0]:
Dpath = '/content/drive/My Drive/Resources/KNBC_v1.0_090925_utf8/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [7]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [8]:
df['label'].value_counts(sort=False)

Sports     22
Keitai     79
Gourmet    57
Kyoto      91
Name: label, dtype: int64

# 文書分類

## 学習データの作成

文章をtfidfベクトルへ

In [0]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [10]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df.values[:,1])
X = df.values[:,0]
print(len(X))
print(X)

249
['烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に 行っ て き た 。 おかき や せんべい の 店 な の だ が 、 これ が オイシイ 。 のれん を くぐる と 小さな 庭 が あり 、 その 先 に 町屋 風 の 店内 が ある 。 せんべい の 箱 は デパート みたい な 山積み で は なく 、 間隔 を あけ て 陳列 さ れ て いる の が また 良い 。 さて 、 この お せんべい 、 普通 に 買う と １５ 枚 入り の 箱 で ７３５ 円 と 高価 な の だ が 、 毎週 木曜 は お 得 な 日 。 割れ たり し て 商品 に なら ない 製品 の まじっ た 「 無 選別 」 販売 が ある の だ 。 一 袋 ２００ ｇ ほど で 価格 は ３００ 円 。 夕方 に は 売切れ て しまう の で 、 欲しい 人 は 早め の 時間 に 買い に 行っ た ほう が いい 。'
 '河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ て き まし た 。 ｈｔｔｐ：／／ｗｗｗ．ｈｏｔｐｅｐｐｅｒ． ｊｐ／Ａ＿ ２０１００ ／ ｓｔｒＪ ００００５６９３７ ． ｈｔｍｌ 居酒屋 、 と 聞く と 酒臭い おじ さん ばかり の イメージ だっ た の です が 、 まったく そんな こと は なく 、 うまい 一 品 料理 と オシャレ な カクテル の 、 雰囲気 の いい 店 な ん です ね 。 結局 同じ よう な 店 を ２ 軒 はしご し まし た が 、 いつ まで で も 話 の ねた は 尽きる こと なく 、 一 晩 中 話し 続け まし た 。 どちら も ２万５千 ぐらい かかり まし た が 、 全部 おごっ て もらい まし た ｗ やっぱり 学生時代 、 いろんな ひと と いろんな 話 を 飽きる ほど し たい もの です 。 この よう な 居酒屋 は 不可欠 です ね 。 おじ さん に なっ て も おそらく は 上司 と の 付き合い や 同僚 と 語る 場 と し て 、 一生 お世話 に なる の か も しれ ませ ん 。 居酒屋 よろしく 。 それ で は また 。'
 'ちょっと 

In [11]:
import re

X
X2 = []
types = []
for x in X:
    now_x = ""
    doc = nlp(x)
    for sent in doc.sents:
        for token in sent:
            # print(token.tag_)
            if not token.tag_ in types:
                types.append(token.tag_)
            # if "名詞" in token.tag_ or "動詞-一般" in token.tag_:
            if "名詞" in token.tag_:
                now_x = now_x+token.lemma_+" "
    now_x = re.sub('\d+', '', now_x).replace("  ", " ")
    now_x = re.sub(r'[a-z]+', "", now_x).replace("  ", " ")
    now_x = re.sub(r'[A-Z]+', "", now_x).replace("  ", " ")
    now_x = re.sub(r'[ａ-ｚ]+', "", now_x).replace("  ", " ")
    now_x = re.sub(r'[Ａ-Ｚ]+', "", now_x).replace("  ", " ")
    # now_x = re.sub(r'[ぁ-ゟ]+', "", now_x).replace("  ", " ")
    # now_x = re.sub(r'[\u30A1-\u30FF]+', "", now_x).replace("  ", " ")    
    print(now_x)
    X2.append(now_x)
print(X2)
print(types)

烏丸 六角 御欠き 屋 さん 六角堂 前 蕪村 庵 御店 御欠き 煎餅 店 此れ 暖簾 庭 有り 先 町屋 風 店内 煎餅 箱 デパート 山積み 間隔 陳列 煎餅 普通 枚 入り 箱 円 高価 毎週 木曜 得 日 割れ 商品 製品 選別 販売 袋 価格 円 夕方 人 時間 買い 
河原町 居酒屋 間 先輩 たち 連れ 河原町 居酒屋 ／＿ 居酒屋 伯父 さん イメージ こと 品 料理 御洒落 カクテル 雰囲気 店 結局 店 軒 梯子 いつ 話 ねた こと 晩 中 どちら 全部 学生時代 人 話 物 居酒屋 伯父 さん 上司 付き合い 同僚 場 一生 御世話 居酒屋 其れ 
贅沢 贅沢 時 何 私 冬季 限定 白薔薇 ココア 俺 買い バンホーテン ココア 時 衝撃 此れ 何 度 衝撃 何 味わい 不思議 白薔薇 牛乳 味 違い 彼れ 特 ミルク 因み オレンジジュース どこ 奴 業務用 業務用 平和 堂 平和 堂 セブン イレブン ホームメイド テースト セブイレ トロピカーナ グルメ 
食欲 秋 最近 御腹 仕方 有り 仕方 有り 私 いつも 昼食 大量 バランス 考 値段 円 反則 中華 御世話 食堂 カツ丼 サラダバー グラム 付け ジェイムズ キッチン 毎回 クーポン 券 御勧め 今 御飯 今日 米 持参 もん 御数 何 考 中 其れ 
季節 限定 月 秋 本番 栗 秋刀魚 茸 旬 味覚 どこ スーパー レストラン 旬 食べ物 季節 限定 メニュー 用意 ハウス 栽培 輸入 主流 現代 季節 限定 味覚 有難み 感じ 所 食べ物 大体 手 此れ いつ 旬 食べ物 便利 時 物 メリット 同時 四季 感覚 此れ 思い 便利 物 所 余談 私 最近 秋 体験 カンフォーラ 友人 ランチ 際 レジ 際 栗 何 気 日 間 御陰 頃 こと 料理 こと こと 体験 
 遍 付近 カフェ 学士 堂 此処 週 回 時 笑い 常連 さん 観光客 気 大学 近く 学生 さん レトロ 庭 有り 此処 居心地 女の子 フード 量 人 本 照明 御洒落 場所 遍 北 右側 味 量 値段 雰囲気 進 堂 教授 大学 人 会話 ， 回 こと 木 机 今 調 人間 国宝 黒田 辰秋 手 樫材 テーブル み 人 外 トイレ 庭 筈 思い 席 此処 フード 量 パンメイン カレー 一度 気 

In [0]:
vectorizer = TfidfVectorizer()
Xv = vectorizer.fit_transform(X2)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(Xv, y, test_size=0.3, random_state=1)
print(X_train.shape)
print(X_train.__class__.__name__)
print(Xv.__class__.__name__)

(174, 4129)
csr_matrix
csr_matrix


In [0]:
Xtrain = torch.Tensor(X_train.todense())
ytrain = torch.LongTensor(y_train)
Xtest = torch.Tensor(X_test.todense())
ytest = torch.LongTensor(y_test)
train = torch.utils.data.TensorDataset(Xtrain, ytrain)
test = torch.utils.data.TensorDataset(Xtest, ytest)

## 3層ネットワーク

In [0]:
input_size = X_train.shape[1]
output_size = len(topics)

class Net(pl.LightningModule):
    def __init__(self, input_size=input_size, hidden_size=10, output_size=output_size, batch_size=10):
        super(Net, self).__init__()
        self.L1 = nn.Linear(input_size, hidden_size)
        # self.dp = nn.Dropout(p=0.3) 
        self.L2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        # x = self.dp(x)
        x = self.L2(x)
        return x
    
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)
        # return torch.optim.Adam(self.parameters())
        
    @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True, num_workers=4)

    def test_dataloader(self):
        # can also return a list of test dataloaders
        return torch.utils.data.DataLoader(test, self.batch_size, shuffle=False, num_workers=4)

    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        test_loss_mean = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': test_loss_mean}

In [146]:
net = Net(hidden_size=10, batch_size=100)
trainer = Trainer(max_nb_epochs=500, gpus=1)
trainer.fit(net)
trainer.test()

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
----------------------------
0 | L1   | Linear | 41 K  
1 | L2   | Linear | 44    


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.5062, device='cuda:0')}
--------------------------------------------------------------------------------



テストデータでの評価

In [149]:
y_pred = []

with torch.no_grad():
    for (docs, labels) in torch.utils.data.DataLoader(test, 1, shuffle=False):
        outputs = net(docs.cuda())
        print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        y_pred.append(predicted.item())

print(classification_report(y_test, y_pred, target_names=topics))

tensor([[-0.3989,  1.2438, -0.3267, -0.3931]], device='cuda:0')
tensor([1], device='cuda:0')
tensor([[ 0.2531, -0.8727,  0.0820,  0.4700]], device='cuda:0')
tensor([3], device='cuda:0')
tensor([[-0.8319, -1.8694,  3.1890, -0.7458]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[-0.8147, -1.7266,  3.4968, -1.2174]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[-0.2370,  0.2679,  0.2469, -0.2744]], device='cuda:0')
tensor([1], device='cuda:0')
tensor([[ 0.0722, -1.2153,  1.7145, -0.7202]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[-1.3244,  1.8615, -0.1388, -0.2267]], device='cuda:0')
tensor([1], device='cuda:0')
tensor([[ 0.2948, -0.8524,  0.0034,  0.4961]], device='cuda:0')
tensor([3], device='cuda:0')
tensor([[-0.4886, -1.9709,  2.9858, -0.7883]], device='cuda:0')
tensor([2], device='cuda:0')
tensor([[ 1.1783, -0.8984,  0.0096, -0.3341]], device='cuda:0')
tensor([0], device='cuda:0')
tensor([[ 0.0382, -0.5456,  0.2319,  0.2365]], device='cuda:0')
tensor

# word2vec

### 参考サイト
* https://github.com/jojonki/word2vec-pytorch/blob/master/word2vec.ipynb
* https://qiita.com/mash0510/items/347964f3eb2e080ea7a4

In [31]:
vocab = {}
vocab_size = 1
for x in df["doc"]:
    doc = nlp(x)
    for sent in doc.sents:
        for token in sent:
            if "名詞" in token.tag_:
                if not token.lemma_ in vocab:
                    vocab[token.lemma_] = vocab_size
                    vocab_size = vocab_size+1

print(vocab)
print(vocab_size)

{'烏丸': 1, '六角': 2, '御欠き': 3, '屋': 4, 'さん': 5, '六角堂': 6, '前': 7, '蕪村': 8, '庵': 9, '御店': 10, '煎餅': 11, '店': 12, '此れ': 13, '暖簾': 14, '庭': 15, '有り': 16, '先': 17, '町屋': 18, '風': 19, '店内': 20, '箱': 21, 'デパート': 22, '山積み': 23, '間隔': 24, '陳列': 25, '普通': 26, '15': 27, '枚': 28, '入り': 29, '735': 30, '円': 31, '高価': 32, '毎週': 33, '木曜': 34, '得': 35, '日': 36, '割れ': 37, '商品': 38, '製品': 39, '選別': 40, '販売': 41, '1': 42, '袋': 43, '200': 44, 'g': 45, '価格': 46, '300': 47, '夕方': 48, '人': 49, '時間': 50, '買い': 51, '河原町': 52, '居酒屋': 53, '間': 54, '先輩': 55, 'たち': 56, '連れ': 57, 'ｊｐ／Ａ＿': 58, '20100': 59, 'strj': 60, '000056937': 61, 'HTML': 62, '伯父': 63, 'イメージ': 64, 'こと': 65, '品': 66, '料理': 67, '御洒落': 68, 'カクテル': 69, '雰囲気': 70, '結局': 71, '2': 72, '軒': 73, '梯子': 74, 'いつ': 75, '話': 76, 'ねた': 77, '晩': 78, '中': 79, 'どちら': 80, '25000': 81, '全部': 82, 'W': 83, '学生時代': 84, '物': 85, '上司': 86, '付き合い': 87, '同僚': 88, '場': 89, '一生': 90, '御世話': 91, '其れ': 92, '贅沢': 93, '時': 94, '何': 95, '私': 96, '冬季': 97, '限定': 98, '白薔薇': 99, 'ココア

In [32]:
datas = []
context_size = 5
for x in df["doc"]:
    doc = nlp(x)
    doc_number = []
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ in vocab:
                doc_number.append(vocab[token.lemma_])
    for i in range(len(doc_number)-context_size):
        datas.append(doc_number[i:i+context_size])
print(datas)
datas_size = len(datas)
print(datas_size)

[[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 10], [7, 8, 9, 10, 3], [8, 9, 10, 3, 11], [9, 10, 3, 11, 12], [10, 3, 11, 12, 13], [3, 11, 12, 13, 14], [11, 12, 13, 14, 15], [12, 13, 14, 15, 16], [13, 14, 15, 16, 17], [14, 15, 16, 17, 18], [15, 16, 17, 18, 19], [16, 17, 18, 19, 20], [17, 18, 19, 20, 11], [18, 19, 20, 11, 21], [19, 20, 11, 21, 22], [20, 11, 21, 22, 23], [11, 21, 22, 23, 24], [21, 22, 23, 24, 25], [22, 23, 24, 25, 1529], [23, 24, 25, 1529, 390], [24, 25, 1529, 390, 11], [25, 1529, 390, 11, 26], [1529, 390, 11, 26, 27], [390, 11, 26, 27, 28], [11, 26, 27, 28, 29], [26, 27, 28, 29, 21], [27, 28, 29, 21, 30], [28, 29, 21, 30, 31], [29, 21, 30, 31, 32], [21, 30, 31, 32, 33], [30, 31, 32, 33, 34], [31, 32, 33, 34, 390], [32, 33, 34, 390, 35], [33, 34, 390, 35, 36], [34, 390, 35, 36, 37], [390, 35, 36, 37, 38], [35, 36, 37, 38, 39], [36, 37, 38, 39, 40], [37, 38, 39, 40, 41], [38, 39, 40, 41, 42], [39, 40, 41, 42, 43], [40, 41

In [33]:
import random

input_word = []
context_word = []
negative_sampling_data = []
for data in datas:
    d = data.copy()
    centor = d.pop(int(context_size/2))
    for i in d:
        input_word.append(centor)
        context_word.append(i)
        negative_sampling_data.append(1)
for data in datas:
    d = data.copy()
    centor = d.pop(int(context_size/2))
    tmp = list(range(vocab_size-1))
    for i in d:
        if i in tmp:
            tmp.remove(i)
    for _ in range(context_size):
        if random.random() < 0.5:
            input_word.append(centor)
            context_word.append(random.choice(tmp))
            negative_sampling_data.append(0)
for i, j, k in zip(input_word, context_word, negative_sampling_data):
    print(i, j, k)
# print(input_word)
# print(context_word)
# print(negative_sampling_data)

ストリーミング出力は最後の 5000 行に切り捨てられました。
4527 4034 0
4577 1650 0
4577 4604 0
2575 3815 0
2575 4003 0
2575 2194 0
4578 838 0
4578 2871 0
4562 2736 0
4562 2432 0
2729 1623 0
2729 3997 0
325 3226 0
3326 504 0
3326 388 0
3326 2979 0
257 3513 0
353 1634 0
748 981 0
748 1725 0
748 1768 0
344 740 0
96 2872 0
96 4614 0
96 2694 0
325 3275 0
325 3897 0
325 3763 0
325 2274 0
4580 4607 0
4580 3245 0
4580 2570 0
207 4553 0
207 118 0
207 2432 0
344 609 0
344 3271 0
344 3886 0
344 4594 0
325 753 0
325 3094 0
325 2648 0
79 2256 0
79 327 0
79 482 0
4527 3196 0
4527 823 0
4527 4148 0
4527 149 0
65 1361 0
65 2067 0
65 2206 0
65 4182 0
65 768 0
724 2156 0
724 2907 0
724 4820 0
65 3793 0
65 2717 0
65 2643 0
325 1274 0
325 946 0
3946 2067 0
3946 3745 0
3946 3734 0
3946 3349 0
65 1216 0
65 1210 0
65 2261 0
4562 186 0
4562 1749 0
4562 3085 0
4562 2829 0
92 396 0
92 4513 0
96 2477 0
155 4796 0
155 428 0
155 3076 0
155 609 0
4521 284 0
4521 2078 0
4521 517 0
54 2317 0
54 935 0
54 1052 0
54 2531 0
2936 2037 0
2936 2286 0

In [0]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(Word2Vec, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
    def forward(self, inputs):
        x = self.embed(inputs)
        return F.relu(x)

In [35]:
from sklearn.utils import shuffle
from torch.autograd import Variable

word2vec_model = Word2Vec(vocab_size, 10)
optimizer = torch.optim.Adam(word2vec_model.parameters(), lr=0.001)
batch_size = 100
loss_fn = nn.MSELoss()

all_losses = []
for epoch in range(100):
    # バッチ化
    batch_input_word = []
    batch_context_word = []
    batch_negative_sampling_data = []
    total_loss = 0
    shuffle_input_word, shuffle_context_word, shuffle_negative_sampling_data = shuffle(input_word, context_word, negative_sampling_data)
    for i in range(0, len(shuffle_input_word), batch_size):
        batch_input_word.append(shuffle_input_word[i:i+batch_size])
        batch_context_word.append(shuffle_context_word[i:i+batch_size])
        batch_negative_sampling_data.append(shuffle_negative_sampling_data[i:i+batch_size])
    # 学習
    for i in range(len(batch_input_word)):
        batch_loss = 0
        tensor_input_word = torch.tensor(batch_input_word[i])
        tensor_input_word = torch.t(tensor_input_word)
        tensor_context_word = torch.tensor(batch_context_word[i])
        out_input_word = word2vec_model(tensor_input_word).view((1, -1))
        out_context_word = word2vec_model(tensor_context_word).view((1, -1))
        score = torch.mm(out_input_word, torch.t(out_context_word))
        log_probs = F.logsigmoid(score)
        loss = loss_fn(log_probs[0], Variable(torch.Tensor([batch_negative_sampling_data[i]])))

        word2vec_model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(epoch, total_loss)
    all_losses.append(total_loss)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1, 100])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1, 52])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 830.2646149396896
1 830.2461526095867
2 830.2646151483059
3 830.1999987065792
4 830.2184611856937
5 830.1815376579762
6 830.2646148204803
7 830.3107691407204
8 830.2738451063633
9 830.2553842365742
10 830.2369216382504
11 830.2830767631531
12 830.2738456428051
13 830.2369230687618
14 830.2553835213184
15 830.2553846240044
16 830.236921787262
17 830.2738459408283
18 830.2553835511208
19 830.2553838193417
20 830.2461536824703
21 830.2923070788383
22 830.2830767333508
23 830.2461537122726
24 830.2553846240044
25 830.2830769121647
26 830.2553851008415
27 830.2738453745842
28 830.3199988603592
29 830.2646141648293
30 830.25538367033
31 830.2369226515293
32 830.3015373945236
33 830.273845821619
34 830.292308151722
35 830.2276910841465
36 830.2553840577602
37 830.3107698261738
38 830.2738456130028
39 830.2830760776997
40 830.2461521327496
41 830.2369226515293
42 830.2646155059338
43 830.301539003849
44 830.2923072874546
45 830.2830756008625
46 830.2646145820618
47 830.2369227409363
48 830.1

In [155]:
print(X_train.shape)
print(vocab)
datas = []
labels = []
label_types = []
label_num_types = {}
doc_num_datas_size = 0
for data in df["doc"]:
    size = 0
    doc = nlp(data)
    for sent in doc.sents:
        for token in sent:
            if token.lemma_ in vocab:
                size += 1
    doc_num_datas_size = doc_num_datas_size if size < doc_num_datas_size else size

zero_tmp = np.array([0.0]*10)
with torch.no_grad():
    for data, label in zip(df["doc"], df["label"]):
        doc = nlp(data)
        doc_num = []
        size = 0
        for sent in doc.sents:
            for token in sent:
                if token.lemma_ in vocab:
                    d = word2vec_model(torch.tensor(vocab[token.lemma_]))
                    d = d.cpu().numpy()
                    doc_num.append(d)
                    size += 1
        for _ in range(doc_num_datas_size-size):
            doc_num.append(zero_tmp)
        # print(np.array(doc_num).reshape((1, -1)).shape)
        # datas.append(np.array(doc_num).reshape((1, -1)).tolist)
        datas.append(doc_num)
        if not label in label_num_types:
            label_num_types[label] = len(label_num_types)
        # l = [0.0]*4
        # l[label_num_types[label]] = 1.0
        labels.append(label_num_types[label])
        if not label in label_types:
            label_types.append(label)
print(np.array(datas).shape)
print(labels)
print(label_types)

(174, 4129)
{'烏丸': 1, '六角': 2, '御欠き': 3, '屋': 4, 'さん': 5, '六角堂': 6, '前': 7, '蕪村': 8, '庵': 9, '御店': 10, '煎餅': 11, '店': 12, '此れ': 13, '暖簾': 14, '庭': 15, '有り': 16, '先': 17, '町屋': 18, '風': 19, '店内': 20, '箱': 21, 'デパート': 22, '山積み': 23, '間隔': 24, '陳列': 25, '普通': 26, '15': 27, '枚': 28, '入り': 29, '735': 30, '円': 31, '高価': 32, '毎週': 33, '木曜': 34, '得': 35, '日': 36, '割れ': 37, '商品': 38, '製品': 39, '選別': 40, '販売': 41, '1': 42, '袋': 43, '200': 44, 'g': 45, '価格': 46, '300': 47, '夕方': 48, '人': 49, '時間': 50, '買い': 51, '河原町': 52, '居酒屋': 53, '間': 54, '先輩': 55, 'たち': 56, '連れ': 57, 'ｊｐ／Ａ＿': 58, '20100': 59, 'strj': 60, '000056937': 61, 'HTML': 62, '伯父': 63, 'イメージ': 64, 'こと': 65, '品': 66, '料理': 67, '御洒落': 68, 'カクテル': 69, '雰囲気': 70, '結局': 71, '2': 72, '軒': 73, '梯子': 74, 'いつ': 75, '話': 76, 'ねた': 77, '晩': 78, '中': 79, 'どちら': 80, '25000': 81, '全部': 82, 'W': 83, '学生時代': 84, '物': 85, '上司': 86, '付き合い': 87, '同僚': 88, '場': 89, '一生': 90, '御世話': 91, '其れ': 92, '贅沢': 93, '時': 94, '何': 95, '私': 96, '冬季': 97, '限定': 98, '白薔

In [156]:
print(np.array(doc_num_datas).shape)
train_datas, test_datas, train_labels, test_labels = train_test_split(datas, labels, test_size=0.3, random_state=1)

(249, 386)


In [0]:
class Network(nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.L1 = nn.Linear(input_size, output_size)
        self.L2 = nn.Linear(output_size, output_size)
        self.L3 = nn.Linear(output_size, output_size)
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        x = F.relu(x)
        x = self.L3(x)
        x = F.sigmoid(x)
        return x

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 3860
output_size = 4
net = Network(input_size, output_size).to(device)

optimizer = torch.optim.Adam(net.parameters())
# optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [223]:
batch_size = 100
for epoch in range(10000):
    all_loss = 0

    batch_datas = []
    batch_labels = []
    shuffle_datas, shuffle_labels = shuffle(train_datas, train_labels)
    for i in range(0, len(labels), batch_size):
        batch_datas.append(datas[i:i+batch_size])
        batch_labels.append(labels[i:i+batch_size])
    for d, l in zip(batch_datas, batch_labels):
        net.zero_grad()
        batch_loss = 0
        tensor_datas = torch.tensor(d, device=device)
        tensor_datas = tensor_datas.view((tensor_datas.shape[0], 1, -1))
        tensor_datas = tensor_datas.float()
        
        out = net(tensor_datas)

        tensor_labels = torch.tensor(l, dtype=torch.long, device=device)
        out = out.view((out.shape[0], -1))
        # F.cross_entropy(input, target)ではtargetは数値
        batch_loss = F.cross_entropy(out, tensor_labels)
        batch_loss.backward()
        optimizer.step()
        all_loss += batch_loss.item()
    print(epoch, all_loss)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0 4.208553314208984
1 4.189132332801819
2 4.1773741245269775
3 4.166343331336975
4 4.154819488525391
5 4.142257928848267
6 4.129821538925171
7 4.117655873298645
8 4.104267597198486
9 4.092496991157532
10 4.08143162727356
11 4.07051408290863
12 4.059646129608154
13 4.049566268920898
14 4.040229320526123
15 4.030834674835205
16 4.0222272872924805
17 4.0149970054626465
18 4.007911801338196
19 4.0013511180877686
20 3.9950575828552246
21 3.989189386367798
22 3.9834766387939453
23 3.9780889749526978
24 3.972819685935974
25 3.967986583709717
26 3.963568925857544
27 3.959400773048401
28 3.955571413040161
29 3.9519543647766113
30 3.9478962421417236
31 3.9443764686584473
32 3.940415620803833
33 3.936909556388855
34 3.933198928833008
35 3.929643988609314
36 3.9258785247802734
37 3.9222933053970337
38 3.918500542640686
39 3.9147238731384277
40 3.9107797145843506
41 3.9069539308547974
42 3.9028841257095337
43 3.899240732192993
44 3.89472496509552
45 3.8905774354934692
46 3.886405348777771
47 3.8815

In [224]:
test_num = 0
a = 0
ans_datas = []
label_datas = []
with torch.no_grad():

    batch_datas = []
    batch_labels = []
    shuffle_datas, shuffle_labels = shuffle(test_datas, test_labels)
    # for data, label in zip(test_datas, test_labels):
        
    for i in range(0, len(labels), batch_size):
        batch_datas.append(datas[i:i+batch_size])
        batch_labels.append(labels[i:i+batch_size])
    
    for i in range(len(batch_labels)):
        tensor_datas = torch.tensor(batch_datas[i], device=device)
        tensor_datas = tensor_datas.view((tensor_datas.shape[0], 1, -1))
        tensor_datas = tensor_datas.float()
        tensor_labels = torch.tensor(batch_labels[i], device=device)

        out = net(tensor_datas.cuda())
        for ans, label in zip(out, tensor_labels):
            ans = ans.cpu().numpy().argmax()
            # print(label.item(), "->", ans)
            ans_datas.append(ans)
            label_datas.append(label)
            if ans == label:
                a += 1
            test_num += 1
result = [i == j for i, j in zip(ans_datas, label_datas)]
print("predict : ", a / test_num)
print(a)


predict :  0.9959839357429718
248


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


* lightning??